- 워드 임베딩 : 단어를 벡터(행렬)로 표현
    - 희소 표현 (원-핫 벡터)
        - 원핫인코딩과 유사, 표현하고자 하는 단어 인덱스 값만 1이고 나머지는 0으로 표현되는 벡터로 표현하는 방법
        - 공간적인 낭비가 심하고 계산 속도가 느림
    - 밀집 표현 (토큰화)
        - 벡터 차원을 분석가가 정한 임의의 값으로 모든 단어 벡터의 차원을 맞춤
        - 희소 표현에 비해 벡터의 차원이 조밀해지므로 밀집 벡터라고 부름
            - 텐서플로우 - 케라스에서는 토큰화된 단어 벡터를 임베딩 층을 통과시켜 RNN, LSTM, GRU를 이용하여 학습이 가능하도록 처리됨

- 텐서플로우 토큰화 객체 : Tokenizer
    - num_words : 인식 가능한 최대 벡터 길이(단어 수)를 지정
    - oov_token : (검증 데이터에) 학습 데이터에 포함되지 않은 단어가 있을 경우 대체할 단어

- 한글의 토큰화
    - 불용어 : 분석에 쓰지 않을 불필요한 단어
        - 해당 불용어를 문장에서 제거할 필요가 있음
    - 형태소 분리를 먼저 진행하고 불용어를 제거한 이후 한글의 토큰화를 진행함

In [9]:
# 패키지 로드
import sys
import warnings
warnings.filterwarnings(action="ignore")

# from hossam.util import *
# from hossam.plot import *
# from hossam.tensor import *

import requests
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# 학습 샘플 데이터
train_text = ["You are the Best", "You are the Nice"]

In [3]:
# 토큰화 객체 생성
tokenizer = Tokenizer(num_words=10, oov_token="<OOV>")
tokenizer

In [4]:
# 토큰화 학습
tokenizer.fit_on_texts(train_text)

# 각 단어에 부여된 인덱스 번호 확인
print(tokenizer.word_index)

{'<OOV>': 1, 'you': 2, 'are': 3, 'the': 4, 'best': 5, 'nice': 6}


In [5]:
# 학습 결과의 적용
train_text = ["We are the Best", "We are the Nice"]
sequences = tokenizer.texts_to_sequences(train_text)
print(sequences)

[[1, 3, 4, 5], [1, 3, 4, 6]]


In [6]:
# 한글 토큰화 샘플 데이터

poem = """
흘러내린 머리카락이 흐린 호박빛 아래 빛난다.
난 유영한다. 차분하게 과거에 살면서 현재의 공기를 마신다.
가로등이 깜빡인다.
나도 깜빡여준다.
머리카락이 흩날린다.
"""

In [7]:
# 불용어 목록 로드
with open('./data/stopwords-ko.txt') as f:
    tmp = f.readlines()
    stopwords = [i.replace("\n", "") for i in tmp]

stopwords[:5]

['가', '가까스로', '가령', '각', '각각']

In [10]:
# Token(형태소) 분리
if sys.platform == "win32":
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
else:
    mecab = Mecab()
    
nouns = mecab.nouns(poem)
print(nouns)

['머리카락', '호박', '빛', '아래', '난', '유영', '과거', '현재', '공기', '가로등', '나', '머리카락']


In [11]:
# 추출된 명사에서 불용어를 제외하여 새로운 리스트 생성
train_text = [x for x in nouns if x not in stopwords]
train_text

['머리카락', '호박', '빛', '아래', '난', '유영', '과거', '현재', '공기', '가로등', '머리카락']

In [12]:
# 텍스트 토큰화
tokenizer = Tokenizer(num_words=len(nouns), oov_token="<OOV>")
tokenizer.fit_on_texts(train_text)
print(tokenizer.word_index)

{'<OOV>': 1, '머리카락': 2, '호박': 3, '빛': 4, '아래': 5, '난': 6, '유영': 7, '과거': 8, '현재': 9, '공기': 10, '가로등': 11}
